<a href="https://colab.research.google.com/github/palakuriswamy/III-H-assigements-/blob/main/AIML_Module_1_Lab_2_Machine_learning_terms_and_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##AIML Module 1 - Lab 2
# Machine learning terms and metrics


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [9]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-9-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [10]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [11]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [12]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [13]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [14]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [15]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [16]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [17]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [18]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [19]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1. How is the accuracy of the validation set affected if we increase the percentage of the validation set? What happens when we reduce it?

    Increasing Validation Set Percentage: When you increase the size of the validation set, you allocate more data for validation, which can lead to a more accurate estimation of model performance on unseen data. However, it reduces the amount of data available for training, which may lead to underfitting.

    Example: If you initially allocate 20% of your data for validation and achieve a validation accuracy of 90%, increasing the validation set size to 30% might result in a slight drop in validation accuracy, say to 88%.

    Reducing Validation Set Percentage: When you reduce the size of the validation set, you allocate more data for training, which can help the model learn better. However, it might lead to a less accurate estimation of model performance on unseen data because you're evaluating it on a smaller validation set.

    Example: If you initially allocate 20% of your data for validation and achieve a validation accuracy of 90%, reducing the validation set size to 10% might result in a slightly higher validation accuracy, say 92%

2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

    Larger Train Set: A larger training set provides more data for the model to learn from, which can lead to better model performance. However, a smaller validation set may result in a less accurate estimate of how the model will perform on the test set because there's less data for validation.

    Larger Validation Set: A larger validation set provides a more reliable estimate of model performance during training. However, if the training set is too small, the model might not generalize well, and the estimate from the validation set could be overly optimistic.

Example: Imagine you have a dataset with 1000 samples. Allocating 75% for training (750 samples) and 25% for validation (250 samples) might provide a reasonable balance between training and validation. In this case, the model will have sufficient data to learn and a reasonable estimate of performance on unseen data.

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X, y = np.random.rand(1000, 10), np.random.randint(0, 2, size=1000)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy with 80% Training Data: {val_accuracy:.2f}")


Validation Accuracy with 80% Training Data: 0.51


3. What do you think is a good percentage to reserve for the validation set so that these two factors are balanced?

The choice of the percentage for the validation set depends on the size of your dataset and the complexity of your model. A common rule of thumb is the 70/30 or 80/20 split for moderate-sized datasets, where 70% or 80% is used for training and the remaining 30% or 20% for validation. However, you can adjust this based on your specific scenario:

    Small Dataset: If you have a small dataset, you might want to allocate a larger percentage for validation (e.g., 30% or more) to ensure a reliable estimate of performance.

    Complex Model: If your model is complex and prone to overfitting, you may allocate a larger validation set to monitor its performance more closely and prevent overfitting.

    Simple Model: For simpler models or large datasets, you can allocate a smaller validation set (e.g., 20%) to ensure you have enough data for training.

Remember that these percentages are not set in stone and should be adjusted based on experimentation and domain knowledge. Cross-validation techniques like k-fold cross-validation can also help obtain more robust performance estimates.

In [27]:
import numpy as np
from sklearn.model_selection import train_test_split

X, y = np.random.rand(1000, 10), np.random.randint(0, 2, size=1000)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")


Training set size: 700 samples
Validation set size: 300 samples


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [21]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [22]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Does averaging the validation accuracy across multiple splits give more consistent results?

  Yes, averaging validation accuracy across multiple splits provides more consistent results. By doing multiple splits and averaging, you reduce the impact of the randomness in a single split. This helps to obtain a more stable estimate of your model's performance.

  example: Imagine you have a coin, and you want to know how likely it is to land heads when you flip it. You decide to flip the coin multiple times to get a more accurate estimate.

In [23]:
import random

def flip_coin():
    return random.randint(0, 1)

num_flips = 10
num_experiments = 5
experiment_results = []

for _ in range(num_experiments):
    flips = [flip_coin() for _ in range(num_flips)]
    head_percentage = sum(flips) / num_flips * 100
    experiment_results.append(head_percentage)

average_head_percentage = sum(experiment_results) / num_experiments
print(f"Average Percentage of Heads Across {num_experiments} Experiments: {average_head_percentage:.2f}%")


Average Percentage of Heads Across 5 Experiments: 48.00%


2.Does it give a more accurate estimate of test accuracy?

  Averaging the validation accuracy across multiple splits provides a more accurate estimate of test accuracy compared to a single split. It provides a better representation of how well your model is likely to perform on unseen data because it considers various combinations of training and validation sets.

In [24]:
import random

split1_accuracy = random.uniform(0.7, 0.9)
split2_accuracy = random.uniform(0.6, 0.8)
split3_accuracy = random.uniform(0.65, 0.85)

average_validation_accuracy = (split1_accuracy + split2_accuracy + split3_accuracy) / 3

print(f"Validation Accuracy (Split 1): {split1_accuracy:.2f}")
print(f"Validation Accuracy (Split 2): {split2_accuracy:.2f}")
print(f"Validation Accuracy (Split 3): {split3_accuracy:.2f}")
print(f"Average Validation Accuracy: {average_validation_accuracy:.2f}")


Validation Accuracy (Split 1): 0.90
Validation Accuracy (Split 2): 0.62
Validation Accuracy (Split 3): 0.65
Average Validation Accuracy: 0.72


3.What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

  Increasing the number of iterations generally leads to a better estimate of test accuracy. With more iterations, you sample from a wider range of possible training and validation sets, which improves the robustness of your estimate. However, there may be diminishing returns, and the improvement in accuracy may become less significant as the number of iterations increases.

In [25]:
import random

def estimate_accuracy(num_iterations):
    average_validation_accuracy = sum(random.uniform(0.7, 0.9) for _ in range(num_iterations)) / num_iterations
    return average_validation_accuracy

num_iterations_list = [10, 50, 100, 500, 1000]

for num_iterations in num_iterations_list:
    avg_accuracy = estimate_accuracy(num_iterations)
    print(f"Avg Accuracy (Iterations={num_iterations}): {avg_accuracy:.2f}")


Avg Accuracy (Iterations=10): 0.81
Avg Accuracy (Iterations=50): 0.79
Avg Accuracy (Iterations=100): 0.79
Avg Accuracy (Iterations=500): 0.80
Avg Accuracy (Iterations=1000): 0.80


4.Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

  Increasing the number of iterations can help mitigate the impact of a small training or validation dataset to some extent. It allows you to explore different combinations of data splits, potentially uncovering patterns that may not be apparent in a single split. However, there are limits to how much increasing iterations can compensate for a very small dataset. If your dataset is extremely small, it's generally advisable to try to obtain more data if possible, as increasing iterations can be computationally expensive

In [26]:
import random

def estimate_accuracy(num_iterations, train_dataset_size):
    validation_accuracies = [random.uniform(0.7, 0.9) for _ in range(num_iterations)]
    avg_accuracy = sum(validation_accuracies) / num_iterations
    adjusted_accuracy = avg_accuracy * (train_dataset_size / 1000)
    return avg_accuracy, adjusted_accuracy

train_dataset_size = 100
num_iterations_list = [10, 50, 100, 500, 1000]

for num_iterations in num_iterations_list:
    avg_acc, adj_acc = estimate_accuracy(num_iterations, train_dataset_size)
    print(f"Avg Accuracy (Iterations={num_iterations}): {avg_acc:.2f}, Adjusted Accuracy: {adj_acc:.2f}")


Avg Accuracy (Iterations=10): 0.81, Adjusted Accuracy: 0.08
Avg Accuracy (Iterations=50): 0.79, Adjusted Accuracy: 0.08
Avg Accuracy (Iterations=100): 0.79, Adjusted Accuracy: 0.08
Avg Accuracy (Iterations=500): 0.80, Adjusted Accuracy: 0.08
Avg Accuracy (Iterations=1000): 0.80, Adjusted Accuracy: 0.08
